This script will apply the extraction from the zip file of downloaded images from Planet; Prepare the images creating the PNG images from the tif file allowing for the manual annotation; and will segment the images describe them for check, and cull them to suit the model requirements, and finally apply the model training. 

First. we define the parth of the project, and refer the python code to the correct python scripts containing the functions that are automatically called when running dependencies. 

In [1]:
import sys
import os
import yaml

# Add the project root to sys.path (adjust as needed)
sys.path.append(os.path.abspath("counting_waterholes"))


import counting_boats.boat_utils.planet_utils as planet_utils
import counting_boats.boat_utils.testing as testing

c:\Users\fossatia\AppData\Local\miniconda3\envs\Boats\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
YOLOv5  v7.0-394-g86fd1ab2 Python-3.10.16 torch-1.12.1+cu113 CUDA:0 (GeForce GTX 1080, 8192MiB)

Fusing layers... 
Model summary: 157 layers, 7015519 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 


We can now extract the file called "composite.tif" obtained from the planet order and downloaded into the zip file in our "raw_images" folder. It will render a tif file and will rename it with the date_aoi.tif outside the zip file. 

In [ ]:
import os
import yaml
import counting_boats.boat_utils.planet_utils

# Define the path to your zip file
zip_path = "images/raw_images" 
#AFUN: should this path be training/raw_images? 
#If so, need to be careful where we download the image zip file. Might need to adaot this. 

# Run extraction
counting_boats.boat_utils.planet_utils.extract_zip(zip_path)

We will now transform the raw tif image into a usable png for future steps. It creates a padded png image to exactly match the size dividable by the stride and tile size.   
Caution that you need to define the config file called "config_train_Drive_UN" as instructed to make sure it matches your paths and runns everything smoothly. 

In [ ]:
import os
import yaml
import counting_boats.train

#Run preparation of the tif files into png and renamed the tif. 
counting_boats.train.prepare("config_train_Drive.yaml")
 

Doing gdal work...
Done with gdal work for 20241103_mimal_test.tif
New Width:  14976 New Height:  12064
Processed 1/1 images


Once the png is created, as we are in the training of the model phase, we need to label the training image. In order to do so, you need to use LabelMe.  
LabelMe is called in your terminal, manually typing "labelme".   
You have to then annotate the waterholes on your padded png image. This creates at the end a json file with all my bounding boxes definitions. Those labels will be needed now to segment the image and the corresponding labels for training purposes. 

Once the whole manual annotation is done, save the outputs, and come back to this script to run the segmentation of the padded png image you just labelled.

In [ ]:
import os
import yaml
import counting_boats.train

#segment the png images
counting_boats.train.segment("config_train_Drive.yaml", train_val_split=0.8)


{'yolo_dir': 'C:/Users/fossatia/Documents/Waterholes_project/yolov5', 'python': 'python', 'raw_images': 'D:\\Waterholes_project\\counting_waterholes\\training_v4\\extra_tif', 'proj_root': '.', 'output_dir': 'D:\\Waterholes_project\\counting_waterholes\\training_v4\\pngs', 'segmented_images': 'results\\images\\train', 'segmented_labels': 'results\\labels\\train', 'weights': './data/NN_weights.pt', 'cache': False, 'workers': 6, 'BATCH_SIZE': 8, 'EPOCHS': 500, 'img_size': 416, 'img_stride': 104, 'TILE_SIZE': 416, 'STRIDE': 104, 'path': '.', 'pngs': '.\\images\\pngs', 'labels': '.\\training\\labels', 'classifications': '.\\images\\classifications', 'training_path': '.\\training', 'train': '.\\training\\images', 'val': None, 'names': {0: 'Dry_WH', 1: 'WH_swamp', 2: 'WH_wet', 3: 'WH_sink', 4: 'U'}}
['D:\\Waterholes_project\\counting_waterholes\\training_v4\\pngs\\20230428_example.json', 'D:\\Waterholes_project\\counting_waterholes\\training_v4\\pngs\\20240101_mimal_test.json', 'D:\\Waterhole

Saving Segments: 100%|██████████| 5673/5673 [00:30<00:00, 187.31it/s]


Skipped 4259 images
Empty 4060 images
Cropping Image: D:\Waterholes_project\counting_waterholes\training_v4\pngs\20240101_mimal_test.png
[12064 14976     3]
We will have:  15933  images maximum
90.0% of images without labels will be removed


Saving Segments: 100%|██████████| 15933/15933 [01:17<00:00, 204.65it/s]


Skipped 13135 images
Empty 13857 images
Cropping Image: D:\Waterholes_project\counting_waterholes\training_v4\pngs\20240204_mimal_test.png
[12064 14976     3]
We will have:  15933  images maximum
90.0% of images without labels will be removed


...

Skipped 13005 images
Empty 13594 images
Cropping Image: D:\Waterholes_project\counting_waterholes\training_v4\pngs\20241103_mimal_test.png
[12064 14976     3]
We will have:  15933  images maximum
90.0% of images without labels will be removed


Saving Segments: 100%|██████████| 15933/15933 [01:22<00:00, 192.70it/s]


Skipped 13064 images
Empty 13825 images
Cropping Image: D:\Waterholes_project\counting_waterholes\training_v4\pngs\20250205_NT.png
[ 8736 12064     3]
We will have:  9153  images maximum
90.0% of images without labels will be removed


Saving Segments: 100%|██████████| 9153/9153 [00:47<00:00, 193.17it/s]


Skipped 8026 images
Empty 5759 images


After segmentation, we evaluate the results of the segmentation and production of material to train the model using the "train.describe" function. 
Run the bellow cell to describe from created paths of segmented images. 

In [ ]:
import sys
import os
import yaml
import counting_boats.train

#describe the created segmented images: 
counting_boats.train.describe("config_train_Drive.yaml")


Config path: D:\Waterholes_project\counting_waterholes\training_v4\output\images


KeyboardInterrupt: 

Apply the cull command which will remove images with no labels until 10% of the training set has no labels. This has to be done post segmentation as we don't know prior the the amount (depends on the segmentation). 
Using my created function as the official cull from Charlie doesn't work well. Mine runs all good. 

In [ ]:
import sys
import os
import yaml
import random
import shutil
from pathlib import Path 
import counting_boats.train

#describe the created segmented images: 
counting_boats.train.cull_AF("config_train_Drive.yaml")


Analyzing label files in: D:\Waterholes_project\counting_waterholes\training_v4\output\labels\train
Looking for corresponding images in: D:\Waterholes_project\counting_waterholes\training_v4\output\images\train
Total label files found: 13565
Empty label files found: 6063
Non-empty label files: 7502
Moving 5230 empty label files to maintain 10% ratio

--- SUMMARY ---
Total label files moved: 5230
Total image files moved: 5230
Remaining total label files: 8335
Remaining empty label files: 833
Empty labels now make up 9.99% of the dataset
Empty labels moved to: D:\Waterholes_project\counting_waterholes\training_v4\output\labels\moved_empty_labels
Corresponding images moved to: D:\Waterholes_project\counting_waterholes\training_v4\output\images\moved_empty_images

SUCCESS: Empty labels now make up 10% or less of the dataset.


Once the cull function is applied reducing the no instance images amount to 10%, we can try to train the model. 
But first let's reorganise the folders to be used in the model. 

In [ ]:
import sys
import os
import yaml
import counting_boats.train

counting_boats.train.reorganize_folders("config_train_Drive.yaml")

Copying from D:\Waterholes_project\counting_waterholes\training_v4\output\images\val to D:\Waterholes_project\counting_waterholes\training_v4\train\val\images
Successfully copied to D:\Waterholes_project\counting_waterholes\training_v4\train\val\images
Copying from D:\Waterholes_project\counting_waterholes\training_v4\output\images\train to D:\Waterholes_project\counting_waterholes\training_v4\train\train\images
Successfully copied to D:\Waterholes_project\counting_waterholes\training_v4\train\train\images
Copying from D:\Waterholes_project\counting_waterholes\training_v4\output\labels\val to D:\Waterholes_project\counting_waterholes\training_v4\train\val\labels
Successfully copied to D:\Waterholes_project\counting_waterholes\training_v4\train\val\labels
Copying from D:\Waterholes_project\counting_waterholes\training_v4\output\labels\train to D:\Waterholes_project\counting_waterholes\training_v4\train\train\labels
Successfully copied to D:\Waterholes_project\counting_waterholes\trainin

Modifiy now in the config file of the yolo model training the path directory of the run. then run the following code and go in the yolov5 folder to run it and train the model. 

This code provides you with the command to excecute in the cmd of the yolo.  

In [ ]:
import sys
import os
import yaml
import counting_boats.train

#describe the created segmented images: 
counting_boats.train.train("config_train_Drive.yaml")

python C:/Users/fossatia/Documents/Waterholes_project/yolov5/train.py --device cuda:0 --img 416 --batch 8 --workers 6 --epochs 100 --data config_train_Drive.yaml --weights C:/Users/fossatia/Documents/Waterholes_project/yolov5/runs/train/exp3/weights/best.pt --save-period 50


Training of the model when ordered directly into the cmd panel and not via notebook: 
Need to figure out how to do it from here to streamline the whole process though...

In [ ]:
python train.py --workers 2 --img 416 --batch 8 --epochs 150 --data config_train_GPU_yolo.yaml --weights yolov5s.pt --cache disk

So it actually seems that it runs automatically with the function train but we do not see any progression... 
Prefer for now to run it manually in the cmd of the yolov5 folder.  

Changed the cache to the SSD drive we are using as we are limited in the storage available locally. 
Needed to set the project to the SSD which saves the outputs and doesn't increase the C: storage usage. 

Need to actually create the D:/temp and D:/yolo_run on your Drive or external directory 

In [ ]:
set TMPDIR=D:/temp
set TEMP=D:/temp
set TMP=D:/temp
set KMP_DUPLICATE_LIB_OK=TRUE 
python C:/Users/fossatia/Documents/Waterholes_project/yolov5/train.py --device cuda:0 --img 416 --batch 4 --workers 2 --epochs 50 --data C:\Users\fossatia\Documents\Waterholes_project\counting_waterholes\config_train_Drive.yaml --weights C:/Users/fossatia/Documents/Waterholes_project/yolov5/runs/train/exp3/weights/best.pt --cache False --project D:/yolo_runs

The set KMP_DUPLICATE_LIB_OK=TRUE is not recommended on the error command... I tried to google it and it seems we should force an install of the Nomkl using 'conda install nomkl --channel conda-forge'. 
However, by doing so, dependencies might be altered. To be checked. 

End of this script. 